In [1]:
!pip install prophet --quiet

In [1]:
import pandas as pd
from prophet import Prophet
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import r2_score

#forecast settings
forecast_steps = 17280  #6 months of 15-min intervals
cell_col = "NCI"
timestamp_col = "Timestamp"

#forecasting for a single KPI across all cells
def forecast_with_prophet(df,kpi_col,output_path):
    all_forecasts=[]
    accuracy_records=[]

    for cell in df[cell_col].unique():
        print(f"\n Forecasting for cell {cell} | KPI: {kpi_col}")
        cell_df=df[df[cell_col] == cell][[timestamp_col, kpi_col]].dropna()

        #converting timestamp to datetime
        try:
            cell_df[timestamp_col] = pd.to_datetime(cell_df[timestamp_col],format="%m-%d-%Y %H:%M:%S",errors='coerce')
            cell_df = cell_df.dropna().sort_values(timestamp_col)

        except Exception as e:
            print(f"Failed to parse timestamps for cell {cell}: {e}")
            continue

        #prophet requires columns:ds(datetime) and y(value)
        ts=cell_df.rename(columns={timestamp_col:"ds", kpi_col:"y"})

        #fit prophet model
        model=Prophet(daily_seasonality=True,weekly_seasonality=True)
        model.fit(ts)
        forecast_full = model.predict(ts[["ds"]])
        accuracy = r2_score(ts["y"],forecast_full["yhat"]) * 100
        print(f"Accuracy for cell {cell}:{accuracy:.2f}%")
        accuracy_records.append({
            "Cell": cell,
            "KPI": kpi_col,
            "Accuracy(%)": round(accuracy,2)
        })

        #forecast for next 6 months starting from May 2,2025
        future_start = pd.Timestamp("2025-05-02 00:00:00")
        future_end = future_start + pd.Timedelta(minutes=15 * forecast_steps)
        future = pd.date_range(start=future_start,end=future_end,freq="15min")
        future = pd.DataFrame({"ds":future})
        forecast = model.predict(future)
        forecast = forecast[forecast["ds"] > ts["ds"].max()]

        #preparing output
        forecast["nCI"]=cell
        forecast["KPI"]=kpi_col
        forecast.rename(columns={"ds":"Timestamp", "yhat":"Forecast"},inplace=True)
        all_forecasts.append(forecast[["Timestamp","Forecast","nCI","KPI"]])

    #combine and save
    final_df=pd.concat(all_forecasts,ignore_index=True)
    final_df.to_csv(output_path,index=False)
    print(f"\n Forecasts saved to:{output_path}")

    # Save accuracy
    accuracy_df=pd.DataFrame(accuracy_records)
    accuracy_output_path = output_path.replace(".csv","_accuracy.csv")
    accuracy_df.to_csv(accuracy_output_path,index=False)
    print(f"Accuracy report saved to:{accuracy_output_path}")

#loading data
df_prb=pd.read_csv("/content/drive/MyDrive/DL_Prb_Utilization_Data.csv")
df_ue=pd.read_csv("/content/drive/MyDrive/Avg_UE_Number_Data.csv")

#run forecasting for both KPIs
forecast_with_prophet(df_prb,"DL_Prb_Utilization","/content/drive/MyDrive/forecast_DL_Prb_Utilization_Prophet.csv")
forecast_with_prophet(df_ue,"Avg_UE_Number","/content/drive/MyDrive/forecast_Avg_UE_Number_Prophet.csv")


 Forecasting for cell 357783981 | KPI: DL_Prb_Utilization


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpyk_ulngh/hnld6zi3.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpyk_ulngh/9_6tk_pg.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=50271', 'data', 'file=/tmp/tmpyk_ulngh/hnld6zi3.json', 'init=/tmp/tmpyk_ulngh/9_6tk_pg.json', 'output', 'file=/tmp/tmpyk_ulngh/prophet_model8_nz8akg/prophet_model-20250531144245.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
14:42:45 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
14:42:50 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Accuracy for cell 357783981:81.35%

 Forecasting for cell 357783979 | KPI: DL_Prb_Utilization


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpyk_ulngh/_qf6wsfg.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpyk_ulngh/qys6xbqc.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=2894', 'data', 'file=/tmp/tmpyk_ulngh/_qf6wsfg.json', 'init=/tmp/tmpyk_ulngh/qys6xbqc.json', 'output', 'file=/tmp/tmpyk_ulngh/prophet_modelum9av3m5/prophet_model-20250531144302.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
14:43:02 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
14:43:06 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Accuracy for cell 357783979:81.30%


...

Accuracy for cell 358531244:74.15%

 Forecasting for cell 358531245 | KPI: Avg_UE_Number


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpyk_ulngh/cpxdlip9.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpyk_ulngh/nmk8ue_x.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=86505', 'data', 'file=/tmp/tmpyk_ulngh/cpxdlip9.json', 'init=/tmp/tmpyk_ulngh/nmk8ue_x.json', 'output', 'file=/tmp/tmpyk_ulngh/prophet_model63h5qt4f/prophet_model-20250531144514.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
14:45:14 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
14:45:21 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Accuracy for cell 358531245:74.40%

 Forecasting for cell 358531243 | KPI: Avg_UE_Number


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpyk_ulngh/4v9ndtl3.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpyk_ulngh/b6pofig9.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=3071', 'data', 'file=/tmp/tmpyk_ulngh/4v9ndtl3.json', 'init=/tmp/tmpyk_ulngh/b6pofig9.json', 'output', 'file=/tmp/tmpyk_ulngh/prophet_modellu5pzfve/prophet_model-20250531144530.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
14:45:30 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
14:45:35 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Accuracy for cell 358531243:73.91%

 Forecasts saved to:/content/drive/MyDrive/forecast_Avg_UE_Number_Prophet.csv
Accuracy report saved to:/content/drive/MyDrive/forecast_Avg_UE_Number_Prophet_accuracy.csv
